# Materializing a box

- here is a demonstration of how a box (given the context of the edge in which it is used, i.e. the variables of source and target nodes) has its transition relation split such that every transitions going through a materialized variable are extracted and explicitly put into the transition relation, along with a few corrections that make sure the semantics of the box (within the used context) are not disrupted
- such a box with explicit transitions with the needed variable can be transformed into a so-called `Materialization Recipe` - a data structure resembling a small part of an ABDD, with symbolic names for variables (during the actual Recipe traversal, symbolic names will be replaced with concrete variables)

- it is similar to modifying a code of for-loop such that one particular iteration is computed outside of the for loop (i.e. there is a for-loop before this code and a for-loop after this code)

```py
# change this:
for i in range(0, 10):
    do_something_for(i)

# into this:
for i in range(0, 5):
    do_something_for(i)
do_something_for(5)
for i in range(6, 10):
    do_something_for(i)
# but on the tree automaton's transition relation
```

In [ ]:
from apply.materialization.abdd_pattern import ABDDPattern, MaterializationRecipe
from apply.materialization.box_materialization import create_materialized_box, compute_variable_ranges
from apply.materialization.pattern_finding import abdd_subsection_create, get_state_sym_lookup
from tree_automata.automaton import TTreeAut
from helpers.utils import box_catalogue
from formats.render_dot import convert_to_dot

In [ ]:
box = box_catalogue["LPort"]
convert_to_dot(box)

In [ ]:
invar: int = 1  # variable of the source node
matvar: int = 2  # variable which should be in the materialized nodes
outvars: list[int] = [6, 5]  # variables of the target nodes of the edge
leafvar: int = 20  # pseudo-variable of the leaf nodes
outvar_map = {s: outvars[i] for i, (p, s) in enumerate(box.get_port_order())}
ranges = compute_variable_ranges(box, invar, outvar_map, leafvar)
print(f"state: [ lower bound variable,  upper bound variable ]")
for state, (minvar, maxvar) in ranges.items():
    print(f"{state}: [ {minvar}, {maxvar} ]")

In [ ]:
aut: TTreeAut = create_materialized_box(box, invar, matvar, outvars, leafvar)
print(aut)
convert_to_dot(aut)

In [ ]:
state_sym_lookup: dict[str, str] = get_state_sym_lookup([f"out{i}" for i in range(len(outvars))], aut)
recipe = abdd_subsection_create(state_sym_lookup, aut)
print(recipe)